# The Vikings Network

In [13]:
import numpy as np
import sciris as sc
import starsim as ss

ss_float_ = ss.dtypes.float
ss_int_ = ss.dtypes.int


In [14]:

class VikingsNet(ss.DynamicNetwork):
    """
    Vikings Network
    """
    def __init__(self, key_dict=None, preBoard=True, postBoard=False, **kwargs):
        """
        Initialized empty and filled with vikings arrival throughout the simulation
        """
        key_dict = sc.mergedicts(dict(dur=ss_float_, start=ss_int_, end=ss_int_), key_dict)
        super().__init__(key_dict=key_dict, preBoard=preBoard, postBoard=postBoard, **kwargs)
        return

    def init_pre(self, sim):
        " Initialize the network "
        super().init_pre(sim)
        return
    
    def add_pairs(self, viking_inds=None, second_viking_inds=None, dur=None, start=None):
        " Add connections between vikings and newly arrived vikings to pair them into ship's doorms"
        if viking_inds is None:
            return 0
        else:
            if start is None:
                start = np.full_like(dur, fill_value=self.sim.ti)
            n = len(viking_inds)
            beta = np.ones(n)
            end =  start + 2 
            self.append(p1=viking_inds, p2=second_viking_inds, beta=beta, dur=dur, start=start, end=end)
            return n
        
    def update(self):
        """
        Set beta to 0 for vikings who complete duration of disease transmission
        """
        inactive = self.edges.end <= self.sim.ti
        self.edges.beta[inactive] = 0
        return

    def end_pairs(self):
        people = self.sim.people
        active = (self.edges.end > self.sim.ti) & people.alive[self.edges.p1] & people.alive[self.edges.p2]
        for k in self.meta_keys():
            self.edges[k] = self.edges[k][active]
        return len(active)
        


In [15]:
pars = dict(
    diseases = ss.SIR(unit='day', dt=1.0, init_prev=0.1, beta=ss.beta(0.01)),
    demographics = ss.Births(unit='year', dt=0.25),
    start = '2000',
    # networks = ss.RandomNet(unit='week'), # We are going to use our own network. 
    n_agents = 500,
)
sim = ss.Sim(pars, unit='day', dt=2, start='2000-01-01', stop='2002-01-01')
sim.init()


Initializing sim with 500 agents


Sim(n=500; 2000-01-01—2002-01-01; demographics=births; diseases=sir)

In [16]:

my_network = VikingsNet()
my_network.init_pre(sim)    # Register the network with the simulation object ( this is calling module.init_pre() - network is a module - to connect the network to the sim)
my_network.add_pairs(viking_inds=[1, 2, 3], second_viking_inds=[100, 101, 102], dur=[5, 1, 1])
net = sc.objdict( my_network=my_network)

print(net)


#0. 'my_network':
vikingsnet("vikingsnet", p1, p2, beta, dur, start, end, preBoard, postBoard)
   p1   p2  beta  dur  start  end
0   1  100   1.0  5.0      0    2
1   2  101   1.0  1.0      0    2
2   3  102   1.0  1.0      0    2


In [17]:

sim.people.alive[100] = False  # Viking dies 
my_network.end_pairs()         # Remove connections from non active vikings.


3

In [18]:
# four more vikings arrive

my_network.add_pairs(viking_inds=[7, 8], second_viking_inds=[103, 104], dur=[2, 2])  
net2 = sc.objdict( my_network=my_network)

print(net2)


#0. 'my_network':
vikingsnet("vikingsnet", p1, p2, beta, dur, start, end, preBoard, postBoard)
   p1   p2  beta  dur  start  end
0   2  101   1.0  1.0      0    2
1   3  102   1.0  1.0      0    2
2   7  103   1.0  2.0      0    2
3   8  104   1.0  2.0      0    2


### Now you can try to use this network on implementing diseases like:


Common diseases among the Vikings, like other historical populations, were largely influenced by their environment, 
lifestyle, diet, and living conditions. Based on archaeological evidence, historical records, 
and modern studies of Viking remains, some of the common diseases and health issues that Vikings may have faced include:

2. Tuberculosis (TB)
Description: A bacterial infection that primarily affects the lungs but can also impact other parts of the body. 
            It spreads through the air and would have been more common in crowded living conditions.
Evidence:   DNA evidence from Viking-era skeletons shows traces of tuberculosis bacteria, indicating its presence in Viking populations.

3. Leprosy (Hansen’s Disease)
Description: A chronic bacterial infection that affects the skin, nerves, and respiratory tract. 
            Leprosy was common in medieval Europe, and there is evidence that it affected the Vikings.
Evidence:   Archaeological finds in Scandinavian settlements reveal skeletal deformities consistent with leprosy, 
            and DNA analysis has confirmed the presence of the disease.

4. Parasites and Worms
Description: Vikings, like other medieval populations, likely suffered from intestinal parasites 
            (e.g., tapeworms, roundworms) due to poor sanitation and the consumption of undercooked meat or contaminated water.
Evidence:   Excavations of Viking latrines and burial sites have uncovered eggs of parasitic worms, 
            indicating that intestinal parasites were common.

8. Infectious Diseases (e.g., Plague)
Description: Vikings likely suffered from a range of infectious diseases, including viral and bacterial 
            infections such as smallpox and plague. Vikings engaged in extensive trade and raiding, which exposed them to diseases from different regions.
Evidence:   DNA from Viking skeletons in Norway has shown evidence of Yersinia pestis, 
            the bacterium responsible for the plague, indicating that the Vikings were exposed to the disease.

Viking Health Practices:
Herbal Medicine: Vikings relied on natural remedies using herbs and plants, many of which were based on local knowledge and passed down through generations.
Rituals and Amulets: Vikings also believed in the protective power of amulets, rituals, and Norse gods to ward off illness and disease, often mixing spiritual and practical approaches to health.
In summary, diseases common among Vikings were influenced by their harsh environment, physical lifestyle, and the close-knit, communal living conditions they often experienced. Many health issues they faced are consistent with what we would expect from a medieval, seafaring population.

